In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install --upgrade ktrain

     |████████████████████████████████| 51kB 3.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp36-none-any.whl size=30534 sha256=7dd2f145cc5d0ac3c37379bd8dda254be19c709d24e33d394404e3d31b6d778b
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp36-none-any.whl size=7912 sha256=ca19b49157c41bdd8cd20c728618dd307c7912367758b00ff9329b91d666fcd8
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=6498da9838a0f5b8afa930c7bd1c19f3395a987f67bf3ae8dc99b44c2ba14627
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 5.6MB/s 
     |████████████████████████████

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [4]:
import keras 
from keras_multi_head import MultiHead
from keras_self_attention import SeqSelfAttention

import ktrain
from ktrain import text as txt

import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Embedding, Convolution1D, Dropout, GRU, Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence



In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/FINAL THESIS PROJECT/codes


/content/drive/MyDrive/Colab Notebooks/FINAL THESIS PROJECT/codes


In [6]:
from features import *
from helper import *

In [7]:
#code_path = os.path.dirname(os.path.realpath(os.getcwd()))
code_path = "/content/drive/MyDrive/Colab Notebooks/FINAL THESIS PROJECT/data"
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000


In [8]:
import pickle
with open(data_path +'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [9]:
text = []

for ses_mod in data2:
    text.append(ses_mod['transcription'])

In [10]:

    
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [11]:
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = data_path + 'glove.42B.300d/glove.42B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2736 unique tokens
/content/drive/MyDrive/Colab Notebooks/FINAL THESIS PROJECT/data/glove.42B.300d/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 90


In [39]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [40]:
tokenizer.tokenize("don't be so judgmental")
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

def tokenize_reviews(text):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

token_tr_X1 = []
for i in text:
  a = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(i))
  token_tr_X1.append(a)

In [48]:
print("Length of token: ", len(token_tr_X1))

x_train_text1 = []
x_train_text1 = sequence.pad_sequences(token_tr_X1, maxlen=500)

print("Length of x_train_text1: ", len(x_train_text1))
print(x_train_text1)

Length of token:  4936
Length of x_train_text1:  4936
[[    0     0     0 ... 20676  1143   119]
 [    0     0     0 ...     0  2814   119]
 [    0     0     0 ...   170  2463   136]
 ...
 [    0     0     0 ...  1231  1136   119]
 [    0     0     0 ...  1174  5656   119]
 [    0     0     0 ...   146  1686   119]]


In [15]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [16]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 100, 34)

In [17]:
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(4936, 200, 189, 1)

In [18]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [19]:
counter = 0
for ses_mod in data2:
    if (ses_mod['id'][:5]=="Ses05"):
        break
    counter+=1
counter

3838

In [49]:
xtrain_tx = x_train_text1[:3838]
xtest_tx = x_train_text1[3838:]
ytrain_tx = Y[:3838]
ytest_tx = Y[3838:]

print(xtrain_tx.shape)
print(xtest_tx.shape)

print(ytrain_tx.shape)
print(xtest_tx.shape)

print(xtrain_tx.shape)

(3838, 500)
(1098, 500)
(3838, 4)
(1098, 500)
(3838, 500)


In [45]:
xtrain_sp = x_train_speech[:3838]
xtest_sp = x_train_speech[3838:]
#xtrain_tx = x_train_text[:3838]
#xtest_tx = x_train_text[3838:]
ytrain_sp = Y[:3838]
ytest_sp = Y[3838:]
print(xtrain_sp.shape)
print(xtest_sp.shape)

print(ytrain_sp.shape)
print(xtest_sp.shape)

print(xtrain_sp.shape)

(3838, 100, 34)
(1098, 100, 34)
(3838, 4)
(1098, 100, 34)
(3838, 100, 34)


In [50]:
from tensorflow.keras.models import Model, Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import*
from tensorflow.keras.layers import Embedding

model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
embedding_layer = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

model_text = LSTM(256, return_sequences=True)(embedded_sequences)
model_text = LSTM(256, return_sequences=False)(model_text)
text_outputs = Dense(256)(model_text)

model_text = Model(sequence_input, text_outputs)


speech_inputs = Input(shape=(100, 34))
model_speech = Flatten()(speech_inputs)
model_speech = Dense(1024)(model_speech)
model_speech = Activation('relu')(model_speech)
model_speech = Dropout(0.2)(model_speech)
speech_outputs = Dense(256)(model_speech)

model_speech = Model(speech_inputs, speech_outputs)



concatenated = concatenate([text_outputs, speech_outputs])

model_combined = Activation('relu')(concatenated)
out = Dense(256)(model_combined)
model_combined = Activation('relu')(out)

out = Dense(4, activation='softmax')(model_combined)

merged_model = Model([sequence_input, speech_inputs], out)
merged_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



#model.compile()
model_speech.summary()
model_text.summary()
merged_model.summary()

print("Model1 Built")

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 100, 34)]         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3400)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_19 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
________________________________________________

In [52]:
hist = merged_model.fit([xtrain_tx,xtrain_sp], ytrain_sp, 
                 batch_size=64, epochs=5, verbose=1, 
                 validation_data=([xtest_tx,xtest_sp], ytest_sp))

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
xtrain_mo = x_train_mocap[:3838]
xtest_mo = x_train_mocap[3838:]

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), input_shape=(200, 189,..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarning: The `Merge` layer is deprecated a

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 3400)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_12 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


In [ ]:
hist = model_combined.fit([xtrain_tx,xtrain_sp,xtrain_mo], ytrain_sp, 
                 batch_size=64, nb_epoch=20, verbose=1, 
                 validation_data=([xtest_tx,xtest_sp,xtest_mo], ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3838 samples, validate on 1098 samples
Epoch 1/20
3838/3838 [==============================] - 109s - loss: 2.3534 - acc: 0.3377 - val_loss: 1.2499 - val_acc: 0.4344
Epoch 2/20
3838/3838 [==============================] - 101s - loss: 1.0247 - acc: 0.5555 - val_loss: 0.9496 - val_acc: 0.6148
Epoch 3/20
3838/3838 [==============================] - 101s - loss: 0.7283 - acc: 0.7113 - val_loss: 0.8120 - val_acc: 0.6794
Epoch 4/20
3838/3838 [==============================] - 100s - loss: 0.5554 - acc: 0.7850 - val_loss: 0.8734 - val_acc: 0.6566
Epoch 5/20
3838/3838 [==============================] - 104s - loss: 0.4523 - acc: 0.8273 - val_loss: 0.9274 - val_acc: 0.6603
Epoch 6/20
3838/3838 [==============================] - 108s - loss: 0.3558 - acc: 0.8658 - val_loss: 1.0546 - val_acc: 0.6576
Epoch 7/20
3838/3838 [==============================] - 109s - loss: 0.3070 - acc: 0.8825 - val_loss: 1.0860 - val_acc: 0.6767
Epoch 8/20
3838/3838 [==============================] - 106s - 

In [ ]:
x_train_mocap2 = x_train_mocap.reshape(-1,200,189)
xtrain_mo = x_train_mocap2[:3838]
xtest_mo = x_train_mocap2[3838:]

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(LSTM(256, return_sequences=True, input_shape=(200, 189)))
model_mocap.add(LSTM(256, return_sequences=False))
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 3400)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_21 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
hist = model_combined.fit([xtrain_tx,xtrain_sp,xtrain_mo], ytrain_sp, 
                 batch_size=64, nb_epoch=20, verbose=1, 
                 validation_data=([xtest_tx,xtest_sp,xtest_mo], ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3838 samples, validate on 1098 samples
Epoch 1/20
3838/3838 [==============================] - 135s - loss: 1.5098 - acc: 0.3794 - val_loss: 1.2127 - val_acc: 0.4572
Epoch 2/20
3838/3838 [==============================] - 132s - loss: 1.0074 - acc: 0.5732 - val_loss: 0.9689 - val_acc: 0.5984
Epoch 3/20
3838/3838 [==============================] - 131s - loss: 0.7328 - acc: 0.7084 - val_loss: 0.9150 - val_acc: 0.6330
Epoch 4/20
3838/3838 [==============================] - 133s - loss: 0.5839 - acc: 0.7723 - val_loss: 0.8540 - val_acc: 0.6730
Epoch 5/20
3838/3838 [==============================] - 132s - loss: 0.4515 - acc: 0.8286 - val_loss: 0.9827 - val_acc: 0.6667
Epoch 6/20
3838/3838 [==============================] - 133s - loss: 0.3672 - acc: 0.8559 - val_loss: 0.9922 - val_acc: 0.6730
Epoch 7/20
3838/3838 [==============================] - 132s - loss: 0.3202 - acc: 0.8757 - val_loss: 1.0905 - val_acc: 0.6730
Epoch 8/20
3838/3838 [==============================] - 133s - 

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent, _time_distributed_dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True, recurrent_dropout = 0.2))
model_text.add(Dropout(0.2))
model_text.add(LSTM(256, return_sequences=False, recurrent_dropout = 0.2))
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(LSTM(128, return_sequences=True, input_shape=(100, 34), recurrent_dropout = 0.2))
model_speech.add(Dropout(0.2))
model_speech.add(AttentionDecoder(128,128))
model_speech.add(Dropout(0.2))
model_speech.add(Flatten())
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 100, 128)          83456     
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
AttentionDecoder (AttentionD (None, 100, 128)          246528    
_________________________________________________________________
dropout_22 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               3277056   
Total params: 3,607,040
Trainable params: 3,607,040
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), input_shape=(200, 189,..., padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: UserWarning: The `Merge` layer is deprecated a

In [ ]:
xtrain_mo = x_train_mocap[:3838]
xtest_mo = x_train_mocap[3838:]

In [ ]:
hist = model_combined.fit([xtrain_tx,xtrain_sp,xtrain_mo], ytrain_sp, 
                 batch_size=64, nb_epoch=20, verbose=1, 
                 validation_data=([xtest_tx,xtest_sp,xtest_mo], ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3838 samples, validate on 1098 samples
Epoch 1/20
3838/3838 [==============================] - 134s - loss: 1.3780 - acc: 0.3619 - val_loss: 1.1874 - val_acc: 0.4818
Epoch 2/20
3838/3838 [==============================] - 134s - loss: 1.0887 - acc: 0.5328 - val_loss: 1.0259 - val_acc: 0.5765
Epoch 3/20
3838/3838 [==============================] - 134s - loss: 0.8234 - acc: 0.6701 - val_loss: 0.9108 - val_acc: 0.6248
Epoch 4/20
3838/3838 [==============================] - 132s - loss: 0.6451 - acc: 0.7603 - val_loss: 1.0393 - val_acc: 0.6211
Epoch 5/20
3838/3838 [==============================] - 133s - loss: 0.5396 - acc: 0.7934 - val_loss: 0.9689 - val_acc: 0.6384
Epoch 6/20
3838/3838 [==============================] - 132s - loss: 0.4569 - acc: 0.8236 - val_loss: 0.9315 - val_acc: 0.6840
Epoch 7/20
3838/3838 [==============================] - 131s - loss: 0.3741 - acc: 0.8551 - val_loss: 1.1224 - val_acc: 0.6321
Epoch 8/20
3838/3838 [==============================] - 133s - 

In [ ]:
hist = model_combined.fit([xtrain_tx,xtrain_sp,xtrain_mo], ytrain_sp, 
                 batch_size=64, nb_epoch=25, verbose=1, 
                 validation_data=([xtest_tx,xtest_sp,xtest_mo], ytest_sp))

Train on 3838 samples, validate on 1098 samples
Epoch 1/10


/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3838/3838 [==============================] - 133s - loss: 0.3763 - acc: 0.8525 - val_loss: 0.9981 - val_acc: 0.6658
Epoch 2/10
3838/3838 [==============================] - 135s - loss: 0.3503 - acc: 0.8606 - val_loss: 1.4561 - val_acc: 0.6184
Epoch 3/10
3838/3838 [==============================] - 134s - loss: 0.3502 - acc: 0.8637 - val_loss: 1.1833 - val_acc: 0.6667
Epoch 4/10
3838/3838 [==============================] - 134s - loss: 0.3165 - acc: 0.8760 - val_loss: 1.0661 - val_acc: 0.6740
Epoch 5/10
3838/3838 [==============================] - 134s - loss: 0.3003 - acc: 0.8796 - val_loss: 1.1590 - val_acc: 0.6694
Epoch 6/10
3838/3838 [==============================] - 134s - loss: 0.2903 - acc: 0.8867 - val_loss: 1.3824 - val_acc: 0.6175
Epoch 7/10
3838/3838 [==============================] - 133s - loss: 0.2639 - acc: 0.8960 - val_loss: 1.2120 - val_acc: 0.6694
Epoch 8/10
3838/3838 [==============================] - 134s - loss: 0.2600 - acc: 0.8999 - val_loss: 1.1252 - val_acc: 0.